<a href="https://colab.research.google.com/github/shenaal1992/StockMarketPrediction/blob/main/binary_classification_machine_learning_models_seq_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This code shows Stock market values prediction for Binary values with input seq length = 7

In [ ]:
# Import the libraries
import math
from pandas_datareader import data as web
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import os



In [ ]:
!pip install yfinance

In [ ]:
import yfinance as fyf
fyf.pdr_override() # <-- Here is the fix

from datetime import datetime
start_date = datetime(2012,1,1)
end_date = datetime(2019,12,17)

#Get the stock quote
df = web.DataReader('AAPL', start = start_date, end = end_date)

#Show the data
df


[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2012-01-03,14.621429,14.732143,14.607143,14.686786,12.482928,302220800
2012-01-04,14.642857,14.810000,14.617143,14.765714,12.550007,260022000
2012-01-05,14.819643,14.948214,14.738214,14.929643,12.689342,271269600
2012-01-06,14.991786,15.098214,14.972143,15.085714,12.821991,318292800
2012-01-09,15.196429,15.276786,15.048214,15.061786,12.801657,394024400
...,...,...,...,...,...,...
2019-12-10,67.150002,67.517502,66.464996,67.120003,65.565033,90420400
2019-12-11,67.202499,67.775002,67.125000,67.692497,66.124245,78756800
2019-12-12,66.945000,68.139999,66.830002,67.864998,66.292755,137310400


In [ ]:
# Defining the training Length

# Create a new dataframe with only Close Column
data = df.filter(['Close'])

#Convert the dataframe to numpy array
dataset = data.values

# Get the number of rows to train the model on
training_data_len = math.ceil(len(dataset)*.8) # take 80% of the data and math.ceil used to round up the number of data

#Printing the number of training data
print(training_data_len)

1602


In [ ]:
# Initialize Scaler
scaler_close = MinMaxScaler()


In [ ]:
# Scaling the data
scaled_data_close = scaler_close.fit_transform(dataset)

In [ ]:
sc_data_close  = []
for i in scaled_data_close:
  sc_data_close.append(i.tolist()[0])


In [ ]:
sc_data_close[0:3]

[0.013197402961684368, 0.014606391211013148, 0.0175327802319844]

In [ ]:
sc_data_close[0]

0.013197402961684368

In [ ]:
input_sequence_length = 7

In [ ]:
sc_data_close[8] , sc_data_close[7]

(0.01866762167747882, 0.01967496604702984)

In [ ]:
# Split the data into x_train and y_train data sets
x_train = []
y_train = []


for i in range(input_sequence_length,len(sc_data_close)):
  x_train.append(sc_data_close[i-input_sequence_length:i])
  y_train.append(int((np.where(sc_data_close[i] > sc_data_close[i-1], 1, 0))))



In [ ]:
x_train[0],y_train[0]

([0.013197402961684368,
  0.014606391211013148,
  0.0175327802319844,
  0.020318903742380545,
  0.019891740017918713,
  0.020854445939997024,
  0.02041453080548289],
 0)

In [ ]:
len(x_train),len(y_train)

(1995, 1995)

In [ ]:
sc_data_close[6:8],x_train[0],y_train[0]

([0.02041453080548289, 0.01967496604702984],
 [0.013197402961684368,
  0.014606391211013148,
  0.0175327802319844,
  0.020318903742380545,
  0.019891740017918713,
  0.020854445939997024,
  0.02041453080548289],
 0)

In [ ]:
scaled_data_close[7:9],x_train[1],y_train[1]

(array([[0.01967497],
        [0.01866762]]),
 [0.014606391211013148,
  0.0175327802319844,
  0.020318903742380545,
  0.019891740017918713,
  0.020854445939997024,
  0.02041453080548289,
  0.01967496604702984],
 0)

In [ ]:
# Import the libraries
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import accuracy_score


In [ ]:
# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=42)


# Support Vector Machine

In [ ]:
param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto']
}

# Create an SVM classifier
svm = SVC()

# Perform grid search to find the best parameters
grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, cv=5)
grid_search.fit(X_train, Y_train)

# Get the best parameters and best score from grid search
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Train the SVM classifier with the best parameters
svm_best = SVC(**best_params)
svm_best.fit(X_train, Y_train)

# Make predictions on the test set
y_pred = svm_best.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(Y_test, y_pred)
print("Best Parameters:", best_params)
print("Best Score:", best_score)
print("Accuracy:", accuracy)

Best Parameters: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}
Best Score: 0.5181583072100313
Accuracy: 0.47619047619047616


# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# Remove Warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Define the parameter grid for grid search
param_grid = {
    'C': [0.1, 1, 10],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}

# Create a logistic regression classifier
logreg = LogisticRegression()

# Perform grid search to find the best parameters
grid_search = GridSearchCV(estimator=logreg, param_grid=param_grid, cv=5)
grid_search.fit(X_train, Y_train)

# Get the best parameters and best score from grid search
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Train the logistic regression classifier with the best parameters
logreg_best = LogisticRegression(**best_params)
logreg_best.fit(X_train, Y_train)

# Make predictions on the test set
y_pred = logreg_best.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(Y_test, y_pred)
print("Best Parameters:", best_params)
print("Best Score:", best_score)
print("Accuracy:", accuracy)

Best Parameters: {'C': 1, 'solver': 'newton-cg'}
Best Score: 0.5150372257053292
Accuracy: 0.5463659147869674


# Random Forest Classifier


In [ ]:
# Import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Define the parameter grid for grid search
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt']
}

# Create a Random Forest classifier
rf = RandomForestClassifier()

# Perform grid search to find the best parameters
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5)
grid_search.fit(X_train, Y_train)

# Get the best parameters and best score from grid search
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Train the Random Forest classifier with the best parameters
rf_best = RandomForestClassifier(**best_params)
rf_best.fit(X_train, Y_train)

# Make predictions on the test set
y_pred = rf_best.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(Y_test, y_pred)
print("Best Parameters:", best_params)
print("Best Score:", best_score)
print("Accuracy:", accuracy)

Best Parameters: {'max_depth': None, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Best Score: 0.5087852664576802
Accuracy: 0.48621553884711777


#LSTM model

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

# Convert x_train and y_train to numpy array for train in keras library (convert in to correct data structure for Keras.)
X1_train, Y1_train, X1_test, Y1_test = np.array(X_train),np.array(Y_train), np.array(X_test),np.array(Y_test)

# LSTM Network expects that input to be three dimensional.
# So we have to convert two dimensional data into Three dimensional data.
# Reshape the data.
X1_train = np.reshape(X1_train,(X1_train.shape[0],X1_train.shape[1],1))

def create_model(units=32):
    model = Sequential()
    model.add(LSTM(units = units, return_sequences = True, input_shape =( X1_train.shape[1],1)))
    model.add(LSTM(units = units, return_sequences = True))
    model.add(LSTM(units=units))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model
# Create a KerasClassifier with tunable hyperparameters
model = KerasClassifier(build_fn=create_model, verbose=0)

# Define the hyperparameters to tune
param_grid = {
    'units': [16, 32, 64]  # Number of units in the LSTM layer
}

# Perform grid search with 5-fold cross-validation
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
grid_result = grid.fit(X1_train, Y1_train, epochs = 50, batch_size = 32)

# Print the best hyperparameters and the corresponding accuracy
print("Best Hyperparameters: ", grid_result.best_params_)
print("Best Accuracy: ", grid_result.best_score_)

# Evaluate the best model on the test set
best_model = grid_result.best_estimator_
accuracy = best_model.score(X1_test, Y1_test)
print("Test Accuracy: ", accuracy)

Best Hyperparameters:  {'units': 64}
Best Accuracy:  0.5169474959373475
Test Accuracy:  0.5463659167289734


# Naive Bayes


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# Create a GaussianNB classifier
model = GaussianNB()

# Define the hyperparameters to tune
param_grid = {
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5]}

# Create GridSearchCV object
grid_search = GridSearchCV(model, param_grid, cv=5)

# Model training and hyperparameter tuning
grid_search.fit(X_train, Y_train)

# Get the best model and predict on test data
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_pred, Y_test)
print('Best parameters:', grid_search.best_params_)
print('Accuracy:', accuracy)

Best parameters: {'var_smoothing': 1e-09}
Accuracy: 0.45864661654135336


# Gradient Boosting Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Define the Gradient Boosting Classifier
model = GradientBoostingClassifier()

# Define the hyperparameter grid for Grid Search
param_grid = {
    'n_estimators': [50, 100, 150],          # Number of boosting stages
    'learning_rate': [0.1, 0.05, 0.01],      # Step size shrinkage
    'max_depth': [3, 4, 5],                  # Maximum depth of the individual trees
}

# Create the Grid Search Cross-Validation object
grid_search = GridSearchCV(model, param_grid, cv=5, n_jobs=-1)

# Model training and hyperparameter tuning
grid_search.fit(X_train, Y_train)

# Get the best model and its hyperparameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Predict on the test set using the best model
y_pred = best_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_pred, Y_test)

print('Best Hyperparameters:', best_params)
print('Best Accuracy:', accuracy)

Best Hyperparameters: {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 50}
Best Accuracy: 0.49874686716791977


# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Define the Decision Tree Classifier
model = DecisionTreeClassifier()

# Define the hyperparameter grid for Grid Search
param_grid = {
    'criterion': ['gini', 'entropy'],        # Splitting criterion
    'max_depth': [None, 5, 10, 20],          # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],         # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4]            # Minimum number of samples required to be at a leaf node
}

# Create the Grid Search Cross-Validation object
grid_search = GridSearchCV(model, param_grid, cv=5, n_jobs=-1)

# Model training and hyperparameter tuning
grid_search.fit(X_train, Y_train)

# Get the best model and its hyperparameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Predict on the test set using the best model
y_pred = best_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_pred, Y_test)

print('Best Hyperparameters:', best_params)
print('Best Accuracy:', accuracy)

Best Hyperparameters: {'criterion': 'entropy', 'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 2}
Best Accuracy: 0.506265664160401


# Artificial Neural Network

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

# Convert x_train and y_train to numpy array for train in keras library (convert in to correct data structure for Keras.)
X1_train, Y1_train, X1_test, Y1_test = np.array(X_train),np.array(Y_train), np.array(X_test),np.array(Y_test)

# Function to create ANN model
def create_model(units=32, activation='relu'):
    model = Sequential()
    model.add(Dense(units=units, activation=activation, input_dim=X1_train.shape[1]))
    model.add(Dense(units=units, activation=activation))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Create a KerasClassifier with tunable hyperparameters
model = KerasClassifier(build_fn=create_model, verbose=0)

# Define the hyperparameters to tune
param_grid = {
    'units': [16, 32, 64],  # Number of units in the hidden layer
    'activation': ['relu', 'tanh']  # Activation function
}

# Perform grid search with 5-fold cross-validation
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
grid_result = grid.fit(X1_train, Y1_train)

# Print the best hyperparameters and the corresponding accuracy
print("Best Hyperparameters: ", grid_result.best_params_)
print("Best Accuracy: ", grid_result.best_score_)

# Evaluate the best model on the test set
best_model = grid_result.best_estimator_
accuracy = best_model.score(X1_test, Y1_test)
print("Test Accuracy: ", accuracy)


Best Hyperparameters:  {'activation': 'relu', 'units': 16}
Best Accuracy:  0.5125528991222381
Test Accuracy:  0.5313283205032349


# Convolution Neural Network

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

# Convert x_train and y_train to numpy array for train in keras library (convert in to correct data structure for Keras.)
X1_train, Y1_train, X1_test, Y1_test = np.array(X_train), np.array(Y_train), np.array(X_test), np.array(Y_test)

# Function to create CNN model
def create_model(filters=32, kernel_size=3):
    model = Sequential()
    model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', input_shape=(X1_train.shape[1], 1)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Create a KerasClassifier with tunable hyperparameters
model = KerasClassifier(build_fn=create_model, verbose=0)

# Define the hyperparameters to tune
param_grid = {
    'filters': [16, 32, 64],     # Number of filters in the Conv1D layer
    'kernel_size': [3, 5, 7]     # Size of the convolutional kernel
}

# Perform grid search with 5-fold cross-validation
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
grid_result = grid.fit(X1_train, Y1_train)

# Print the best hyperparameters and the corresponding accuracy
print("Best Hyperparameters: ", grid_result.best_params_)
print("Best Accuracy: ", grid_result.best_score_)

# Evaluate the best model on the test set
best_model = grid_result.best_estimator_
accuracy = best_model.score(X1_test, Y1_test)
print("Test Accuracy: ", accuracy)

Best Hyperparameters:  {'filters': 32, 'kernel_size': 3}
Best Accuracy:  0.5182014107704163
Test Accuracy:  0.5438596606254578
